### Real, Complex and Symplectic Reflection Groups - March 2023, RUB

## Computational Aspects of Complex Reflection Groups

Götz Pfeiffer - University of Galway

# 1. Algorithms: A Glimpse of CGT

![Trees](images/trees.jpg)

## Algorithms

A good algorithm is short yet powerful.

Like this one-line implementation of the [Euclidean Algorithm](https://en.wikipedia.org/wiki/Euclidean_algorithm) in `python`:
```python
def gcd(a,b): return a if b == 0 else gcd(b, a % b)
```
This translates readily into mathematics:
$$
\gcd(a, b) = \begin{cases}
a, & \text{if } b = 0,\\
\gcd(b, a \bmod b), & \text{else}
\end{cases}
$$
which is helpful for **proving desirable properties** of an algorithm (meaning, termination, correctness, ...).  Also it can serve as a **model for refinements** (for speed, ...) or extensions (any bells and whistles, ...).

Most of my algorithms are simple $3$-step procedures:

1. **Initialize.** (a list)
2. **Loop.**  (over the list and grow it)
3. **Return.**  (the list)

On closer inspection, the list really is a tree ...

##  Tree Traversal: BFS vs DFS

* Suppose we want to visit all the nodes of a (rooted) tree like this (with root node in white):

![e7 tree](images/e7tree.png)

* **Breath First Search** (BFS) and **Depth First Search**  (DFS) provide strategies for doing this in a simple and systematic fashion.
* For this, we assume that each **node** $x$ in the tree knows all its **children** as a list $x$.next.
* Then the two **algorithms** for visiting all nodes of a tree with root $x$ can be described as follows.

![bfs vs dfs](images/bfsdfs.png)

* In practice, BFS can take advantage of dynamic lists in a `for` loop.

In [ ]:
BFS := function(x, visit)
    local   Q,  y,  z;
    Q := [x];
    for y in Q do
        visit(y);
        for z in y.next do
            Add(Q, z);
        od;
    od;
end;

* While DFS, using the function call stack, can be implemented recursively. 

In [ ]:
DFS := function(x, visit)
    local   z;
    visit(x);
    for z in x.next do
        DFS(z, visit);
    od;
end;

* For an application to the above example let's organize the tree as a collection of `node` records, with a component `id` (for their own name, taken from their position in a list `nodes`) and a component `next` ( a list of references to their children).

In [ ]:
nodes := List([1..7], i-> rec(id := i, next := []));;
root := nodes[6];;  parent := [3,4,4,5,6,6,6];;
for i in [1..Length(parent)] do
    if parent[i] <> i then
        Add(nodes[parent[i]].next, nodes[i]);
    fi;
od;
root;

* The visitor simply prints the name of each node it encounters ...

In [ ]:
print := function(x)  Print(x.id, ", ");  end;

* ... BFS and DFS list all the nodes in slightly different orders:

In [ ]:
BFS(root, print);

In [ ]:
DFS(root, print);

* Recall the graph: ![e7 tree](images/e7tree.png)

* The recursive strategy can be modified to suit a specific purpose, e.g., to print a tree as a tree 

In [ ]:
t_indent := 0;;  t_nl := false;;
t_print := function(x)
    local   c;
    if t_nl then
        Print(RepeatedString(" ", t_indent));  t_nl := false;
    fi;
    Print("-", x.id);
    if Length(x.next) = 0 then
        Print("\n");  t_nl := true;
    fi;
    t_indent := t_indent + 2;
    for c in x.next do  t_print(c);  od;
    t_indent := t_indent - 2;
end;

In [ ]:
t_print(root);

### Graph Traversal

* Both BFS and DFS can be applied to a (simple or directed) **graph**.
* The same node then can possibly be reached through different paths.
* Some care needs to be taken to manage these repeat encounters.

###  Applications

* distance between nodes
* shortest paths
* connected components
* ...

## Orbit Algorithms

* **Group actions** are a rich source of graphs.
* Here, the **nodes** of the graph are the elements $x$ of the domain that is acted upon.
* The **edges** of the graph are of the form $x \stackrel{s}{\longrightarrow} x.s$, implicitly given by the action.
* The elements $s$ typically come from a set of **generators** of the acting group.

<div class="alert alert-danger">

**Definition.**
Let $G$ be a group **acting from the right** on a set $X$ via $(x, a) \mapsto x.a$, and suppose that $G = \langle S \rangle$ for 
some $S \subseteq G$. The **action graph** of $X$ and $S$ is the (directed) graph with 
* **vertices:** $x \in X$, and
* **edges:** $x \stackrel{s}{\longrightarrow} x.s$ for $x \in X$, $s \in S$.
</div>

* We formulate and apply a variant of BFS called **orbit algorithm** to compute properties of this action.

### The Orbit Algorithm

* In order to have some groups to play with, provide a list of transpositions of adjacent points.

In [ ]:
transpositions := n -> List([2..n], j-> (j-1,j));

* For example, on $4$ points (which group do they generate?):

In [ ]:
swaps := transpositions(4);

* The plain orbit algorithm is BFS with $x$.next ${} = \{x.s : s \in S\}$.
* The action of $G$ on $X$ is described by an **action function** `under`: calling `under(x, s)` returns $x.s$ (&ldquo;$x$ under $s$&rdquo;).

<div class="alert alert-info">
    
**Orbit Algorithm**
    
* **Input:** a list `aaa` of generating operators, a point `x` of the domain $X$, and an action function `under`. 
* **Output:** the **orbit** $x^G = \{x.a : a \in G\}$ of the point `x` under the action of the group generated by `aaa`.
    
</div>

In [ ]:
orbit := function(aaa, x, under)
    local a, y, z, list;
    list := [x];
    for y in list do
        for a in aaa do
            z := under(y, a);
            if not z in list then Add(list, z); fi;
        od;
    od;
    return list;
end;

* To find the **orbit** of a point $x$ under the group generated by the swaps: apply the orbit algorithm to 
  - (i) the swaps, 
  - (ii) the point $x$, 
  - (iii) the standard action **on points** $(x, a) \mapsto x^a$

In [ ]:
onPoints := function(x, a)  return x^a;  end;

In [ ]:
orbit(swaps, 2, onPoints);

* in GAP, this action is called `OnPoints`:

In [ ]:
orbit(swaps, 3, OnPoints);

In [ ]:
swaps;

### Elements

* To find the **elements** of the group generated by the swaps: apply the orbit algorithm to 
  - (i) the swaps, 
  - (ii) the identity permutation, 
  - (iii) the action **on the right** $(x, a) \mapsto x a$

In [ ]:
onRight := function(x, a)  return x * a;  end;

In [ ]:
orbit(swaps, (), onRight);

* in GAP, this action is called `OnRight`

In [ ]:
orbit(swaps, (), OnRight);

* In CGT, it is customary to represent a group $G = \langle A \rangle$ by a list $A$ of generators (avoiding the need to list all its elements where possible).

* Let's turn the list of generators into a group object, and from now on formulate algorithms in terms of the group. 

In [ ]:
group := GroupWithGenerators(swaps);
GeneratorsOfGroup(group);

* for instance, a function to compute the elements of a group:

In [ ]:
elements := function(group)
    return orbit(GeneratorsOfGroup(group), Identity(group), OnRight);
end;

* test it

In [ ]:
eee := elements(group);

In [ ]:
Length(eee);

### Words in the generators

* In order to express the elements of the group as words in the generators, we introduce an action on words.  
* For this, the generators will be represented by symbols $1,2,3,\dotsc$ (Who is acting? On what?)

In [ ]:
onWords := function(word, s)
    return Concatenation(word, [s]);
end;

* The orbit algorithm will now produce two lists in parallel: the list `list` of elements as before, and a list `words` of corresponding words.
* We now need more control over the lists and make the indices `i` (in `list`) and `k` (in `aaa`) explict.
* The loop over `list` becomes a `while` loop an we must not forget to increment `i`.

In [ ]:
orbit_with_words := function(aaa, x, under)
    local   list,  words,  i,  k,  z;
    list := [x];  words := [[]];  i := 0;
    while i < Length(list) do
        i := i+1;
        for k in [1..Length(aaa)] do
            z := under(list[i], aaa[k]);
            if not z in list then
                Add(list, z);
                Add(words, onWords(words[i], k));
            fi;
        od;
    od;
    return rec(list := list, words := words);
end;

* We now can use this new variant of the orbit algorithm in the same way as before.

In [ ]:
www := orbit_with_words(swaps, (), onRight).words;

## Conjugacy Classes

* With suitable arguments, `OnPoints` also implements the **conjugation** action of a group $G$ on itself.
* We can use the orbit algorithm to compute the conjugacy class of a single element in $G$, and to find the list of all conjugacy classes of $G$.

###  Conjugacy Class

* To find the **conjugacy class** of an element $x$ of a group: apply the orbit algorithm to 
  - (i) the generators of the group, 
  - (ii) the element $x$, 
  - (iii) the on-points-action $(x, a) \mapsto x^a = a^{-1} x a$

In [ ]:
class := function(group, x)
    return Set(orbit(GeneratorsOfGroup(group), x, OnPoints));
end;

* test on a random element

In [ ]:
a := Random(eee);

In [ ]:
class(group, a);

* A conjugacy class really should know its group.
* in GAP, the class of `a` in `G` is denoted (and constructed) as `a^G`.

In [ ]:
cl := a^group;

* Such a class has: **elements**, a **representative**, and an **acting domain** (the group `G`).

In [ ]:
Elements(cl);
Representative(cl);
ActingDomain(cl);

###  Conjugacy Classes

* To find all conjugacy classes of a group, the list of generators itself needs to be closed under conjugation.
* We need a version `orbits` of the orbit algorithm that initializes its **queue with several points**, not just one.

In [ ]:
orbits := function(aaa, xxx, under)
    local a, y, z, list;
    list := ShallowCopy(xxx);
    for y in list do
        for a in aaa do
            z := under(y, a);
            if not z in list then Add(list, z); fi;
        od;
    od;
    return list;
end;

* For example, all conjugates of the swaps (what's this set usually called?)

In [ ]:
orbits(swaps, swaps, onPoints);

* The image of a conjugacy class $x^G$ under right multiplication with a group element $a$ is the conjugacy class $(xa)^G$.  (In what sense is this an action? Is it even well-defined?)

In [ ]:
onClasses := function(x, a)
    return OnRight(Representative(x), a)^ActingDomain(x);
end;

* To find all **conjugacy classes** of a group $G$:
  - we close the set `gens` of generators of $G$ under conjugation, and
  - compute the orbit of the class $1^G$ of the identity under the above right action on classes.

In [ ]:
conjugacyClasses := function(group)
    local gens;
    gens := GeneratorsOfGroup(group);
    return orbit(orbits(gens, gens, OnPoints), Identity(group)^group, onClasses);
end;

In [ ]:
cc := conjugacyClasses(group);

In [ ]:
List(cc, Size);

<div class="alert alert-success">

**Remarks**

* The operators `aaa` need not be invertible, nor do they need to generated a group: the orbit algorithm does not require the use of inverses.
* Neither do they need to generate a finite domain: the orbit algorithm can terminate if the list `aaa` and the orbit are finite.

</div>

* Sometimes, the operators `aaa` generate a monoid.
* A well-known monoid is the power set $2^S$ of a finite set $S$, with set union as its binary operation, generated by the singleton sets $\{s\}$, $s \in S$.

## Subgroups

* Let $G$ be a group.
* The power set $(2^G, \cup)$ is a monoid, generated by the singletons $\{a\}$, $a \in G$, as atoms.
* $2^G$ acts on the subgroups $H$ of $G$ via closure: $H.A = \langle H, A \rangle$.  (Check!)

In [ ]:
onGroups := function(x, a)
    return ClosureGroup(x, a);
end;

* In fact, each subgroup of $G$ lies in the orbit of the trivial subgroup.  (Check!)

In [ ]:
subgroups := function(group)
    return orbit(Elements(group), TrivialSubgroup(group), onGroups);
end;

* Let's apply this to our group, generated by the swaps.

In [ ]:
Size(group);

In [ ]:
subs := subgroups(group);

In [ ]:
Length(subs);

### Conjugacy Classes of Subgroups

* Combining ideas from above we can define the image of a conjugacy classe of subgroups $H^G$ under a singleton $\{a\}$  as the conjugacy class $\langle H, a \rangle^G$. (Is this a well-defined action?)

In [ ]:
onSubgroupClasses := function(x, a)
    return onGroups(Representative(x), a)^ActingDomain(x);
end;

* To compute all **conjugacy classes of subgroups** of $G$, we determine the orbit of the class of the trivial subgroup under $(2^G, \cup)$ with respect to that action.
* In fact, it suffices to consider **zuppos** (**z**yklische **U**ntergruppen von **P**rimzahl**p**otenz-**O**rdnung) as potential generators ...

In [ ]:
Length(Zuppos(group));

In [ ]:
subgroupClasses := function(group)
    return orbit(Zuppos(group), TrivialSubgroup(group)^group, onSubgroupClasses);
end;

In [ ]:
ccs := subgroupClasses(group);

In [ ]:
Length(ccs);

In [ ]:
List(ccs, Size);

In [ ]:
Sum(ccs, Size);

In [ ]:
ccs := subgroupClasses(GroupWithGenerators(transpositions(5)));;
Length(ccs);

## Stabilizer and Transversal

* We can use a variant of the orbit algorithm to determine (and remember), for each point $y$ in the $G$-orbit of $x$, a representative element $t_y \in G$ with $x.t_y = y$.
* This list of representatives will form a **transversal** of the cosets of the stabilizer of $x$ in $G$.
* By Schreier's Theorem, a generating set for the **stabilizer** can be computed from the transversal.

### Transversal

* We initialize an additional list `reps` with the identity element, mapping $x$ to itself.
* Whenever a new coset $z$ is found, as image under $a$ of a coset $y$, we add $t_z := t_y a$ to the list `reps`.

In [ ]:
orbit_with_transversal := function(aaa, x, under)
    local   list,  reps,  i,  k,  z;
    list := [x];  reps := [aaa[1]^0];  i := 0;
    while i < Length(list) do
        i := i+1;
        for k in [1..Length(aaa)] do
            z := under(list[i], aaa[k]);
            if not z in list then
                Add(list, z);
                Add(reps, reps[i] * aaa[k]);
            fi;
        od;
    od;
    return rec(list := list, reps := reps);
end;

In [ ]:
swaps := transpositions(5);;
transversal := orbit_with_transversal(swaps, 2, onPoints);

### Stabilizer

<div class="alert alert-danger">

**Schreier's Theorem.**
Suppose a group $G$, generated by a set $S$,
acts on a set $X$ and that $\{t_y : y \in x^G\}$ is a transversal
of the orbit of $x \in X$.  Then
$$
    \{t_y a t_{y.a}^{-1}: a \in A,\, y \in x^G\}
$$
is a set of generators for the stabilizer $G_x$ of $x$ in $G$.
    
</div>

* In this variant of the orbit algorithm, we also collect the Schreier generators $t_ya t_{y.a}^{-1}$ in a list `stab`.

In [ ]:
orbit_with_stabilizer := function(aaa, x, under)
    local   list,  reps,  stab,  i,  k,  l,  z;
    list := [x];  reps := [aaa[1]^0];  stab := [];  i := 0;
    while i < Length(list) do
        i := i+1;
        for k in [1..Length(aaa)] do
            z := under(list[i], aaa[k]);
            l := Position(list, z);
            if l = fail then
                Add(list, z);
                Add(reps, reps[i] * aaa[k]);
            else   # x^(reps[i] * a) = x^reps[l]
                Add(stab, reps[i] * aaa[k] / reps[l]);
            fi;
        od;
    od;
    return rec(list := list, reps := reps, stab := stab);
end;

In [ ]:
stabilizer := orbit_with_stabilizer(swaps, 2, OnPoints);

## Stabilizer Chain

* Schreier's theorem gives generators of the stabilizer, which can be subjected to further orbit calculations.
* Applying the above ideas along a chain of stabilizers can yield information about the whole group.

### Order of the Group

* By the Orbit-Stabilzer Lemma: $|G| = |x^G| \, |G_x| = |x^G| \, |y^{G_x}| \, |G_{x, y}| = {\dots}$

In [ ]:
sizeOfGroup := function(group)
    local x, orb, stab;
    if group = TrivialSubgroup(group) then return 1; fi;
    x := LargestMovedPoint(group);
    orb := orbit_with_stabilizer(GeneratorsOfGroup(group), x, OnPoints);
    stab := Subgroup(group, Difference(orb.stab, [()]));
    return sizeOfGroup(stab) * Length(orb.reps);
end;

In [ ]:
sizeOfGroup(group);

* Some slightly bigger examples:

In [ ]:
group := Group(transpositions(10));

In [ ]:
sizeOfGroup(group);

In [ ]:
m24:= Group([
  (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23),
  (3,17,10,7,9)(4,13,14,19,5)(8,18,11,12,23)(15,20,22,21,16),
  (1,24)(2,23)(3,12)(4,16)(5,18)(6,10)(7,20)(8,14)(9,21)(11,17)(13,22)(15,19) 
]);;
sizeOfGroup(m24);

###  Random Element

* Random elements of a group: along the same lines as `sizeOfGroup`

In [ ]:
randomGroupElement := function(group)
    local x, orb, stab;
    if group = TrivialSubgroup(group) then return Identity(group); fi;
    x := LargestMovedPoint(group);
    orb := orbit_with_stabilizer(GeneratorsOfGroup(group), x, OnPoints);
    stab := Subgroup(group, Difference(orb.stab, [()]));
    return randomGroupElement(stab) * Random(orb.reps);
end;

In [ ]:
randomGroupElement(m24);

* Check: These random elements are **uniformly distributed**!

In [ ]:
group := Group(transpositions(4));
Collected(List([1..2400], i-> randomGroupElement(group)));

* Also, using similar ideas:
  * **membership** test: $a \in G$?
  * express element as **word in the generators**: $a = s_1 \dotsm s_k$
  * **homomorphisms** (defined on generators): $\phi(a) = \phi(s_1) \dotsm \phi(s_k)$

## Graphs

* With a further small modification, the orbit algorithm can keep track of edges and thus construct the **action graph**.  
* Recall that this is a labelled directed graph, with vertices $x, y \in X$ and edges $x \stackrel{a}{\to} y$, whenever $x.s = y$ for $x, y \in X$, and $s \in S$.  
* We need to decide on a data structure for such graphs.  The simplest, perhaps, is a list of pairs of indices, each representing an edge.

In [ ]:
LoadPackage("jupyterviz");  # thanks: Nathan Carter @bentley.edu
opts := rec(vertexwidth := 12, vertexheight := 12, edgecolor := "#def");

In [ ]:
PlotGraph([[2,4],[2,6],[2,8],[4,4]], opts);

* Let's turn the tree from the beginning into an edge list and plot it.

In [ ]:
edges := List([1..Length(parent)], i -> [i, parent[i]]);
edges[6][2] := 6;
PlotGraph(edges, opts);

* A variant of the orbit algorithm keeps track of the edges in a list `edges`.
* For each newly constructed point $z := y.a$, the simple membership test `z in list` needs to be replaced by finding `Position(list, z)`.
* In GAP, `Position` returns `fail` if the element is not found.

In [ ]:
orbit_with_edges := function(aaa, x, under)
    local   list,  edges,  i,  k,  l,  z;
    list := [x];  edges := [];  i := 0;
    while i < Length(list) do
        i := i+1;
        for k in [1..Length(aaa)] do
            z := under(list[i], aaa[k]);
            l := Position(list, z);
            if l = fail then
                Add(list, z);
                l := Length(list);
            fi;
            Add(edges, [i, l]);
        od;
    od;
    return rec(list := list, edges := edges);
end;

* Examples:

In [ ]:
edges := orbit_with_edges(swaps, 1, OnPoints).edges;

In [ ]:
PlotGraph(Set(edges), opts);

In [ ]:
edges := orbit_with_edges(swaps, [1,2], OnPairs).edges;;
edges := Filtered(Set(edges), x-> x[1] <> x[2]);

In [ ]:
PlotGraph(edges, opts);

In [ ]:
edges := orbit_with_edges(transpositions(6), [1,2], OnSets).edges;;
edges := Filtered(Set(edges), x-> x[1] <> x[2]);

In [ ]:
PlotGraph(Set(edges), opts);

In [ ]:
edges := orbit_with_edges(transpositions(4), (), OnRight).edges;;
edges := Filtered(Set(edges), x-> x[1] <> x[2]);

In [ ]:
PlotGraph(edges, opts);

### Permutations

* The action graph is in fact a directed graph.
* It encodes the permutations induced by the action on the domain.
* For this (and other applications) it will be more convenient to store the edge information in a different format: as lists of `images`.

In [ ]:
orbit_with_images := function(aaa, x, under)
    local   list,  images,  i,  k,  l,  z;
    list := [x];  i := 0;
    images := List(aaa, x -> []);
    while i < Length(list) do
        i := i+1;
        for k in [1..Length(aaa)] do
            z := under(list[i], aaa[k]);
            l := Position(list, z);
            if l = fail then
                Add(list, z);
                l := Length(list);
            fi;
            images[k][i] := l;
        od;
    od;
    return rec(list := list, images := images);
end;

* For example:

In [ ]:
swaps := transpositions(5);
orb := orbit_with_images(transpositions(5), 1, OnPoints);

* `PermList` converts an image list into a permutation (in disjoint cycle form).

In [ ]:
List(orb.images, PermList);

* A more interesting example:

In [ ]:
orb := orbit_with_images(swaps, [1,2], OnSets);;
List(orb.images, PermList);

* When the images are converted into edges, it helps to be able to list entries together with their position in a list.  This can be regarded as the transpose of the usual two-line notation for permutations.

In [ ]:
list_with_index := list -> List([1..Length(list)], i -> [i, list[i]]);

In [ ]:
list_with_index(orb.images[1]);

In [ ]:
edges := Union(List(orb.images, list_with_index));;
PlotGraph(Filtered(edges, x -> x[1] <> x[2]));

## Exercises

* Is Euclid's Algorithm BFS or DFS?  What is the tree behind it, vertices, edges?  Make the tree explicit and use it to formulate the [Extended Euclidean Algorithm](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm), which expresses the gcd $d$ of $a$ and $b$ as a linear combination $d = ax + by$.

* **Pólya Action**.  The symmetric group of degree $n$ acts on the words of length $n$ (over a finite alphabet $A$) by rearranging the letters of a given word.  In GAP, this action is called `Permuted`.  Compute the orbit of the word `"11222"` under the action, find the permutations induced by the action on the list of all words in the orbit, plot the action graph and determine the stabilizer of the word.

In [ ]:
word := [1,1,2,2,2];;  swaps := transpositions(Length(word));;
orb := orbit_with_images(swaps, word, Permuted);;
stab := orbit_with_stabilizer(swaps, word, Permuted);;
orb.list;

*  Show that the partition of a (finite) group into conjugacy classes can be computed as an orbit under right multiplication, provided that the set of generators is closed under conjugation.

* For a finite group $G$, show that the monoid $(2^G, \cup)$ acts on the set of subgroups of $G$, and that each subgroup of $G$ lies in the orbit of the trivial subgroup under this action.

* Show that the conjugacy classes of subgroups of a finite group can be computed as an orbit under the action of its power set.

* Prove Schreier's Theorem.

* Show that the power set $2^A = \{B : B \subseteq A\}$ of a (finite) set $A$ is the orbit of $A$ under the take-away action:
```gap
takeAway := function(set, s)
    return Difference(set, [s]);
end;
```

In [ ]:
Read("syt.g");
set := "piza";
orbit(set, set, takeAway);

* Describe other interesting problems, where the right choice of action turns the solution into an application of the orbit algorithm.

* A **composition** of $n$ is a sequence $c = (c_1, \dots, c_k)$ of positive integers.  Set up a bijection between the set of compositions of $n$ and the power set of $A = \{1,\dots, n{-}1\}$ (so that $A$ corresponds to the composition $(n)$, and the subset relationship on $2^A$ corresponds to refinement of compositions).  Use the bijective correspondence to compute the set of all compositions of $n$ as orbit of $(n)$ under a suitable take-away action.

* A **partition** of $n$ is a composition $\lambda = (l_1, \dots, l_k)$ where $l_1 \geq \dots \geq l_k$.  Thus, sorting the parts of any composition in decreasing order yields a partition.  In that sense, a partition is a canonical representative of a rearrangement class of compositions.  Compute the partitions of $n$ as orbit of the partition $(n)$ under a suitable action on canonical composition rearrangement class representatives.

In [ ]:
Read("syt.g");
p4:= partitions(4);

* Formulate a version of BFS that, for a given vertex $x$ in a simple connected graph $\Gamma$, finds a **shortest path** to any vertex $y$ in $\Gamma$.

* Formulate an algorithm that, for given vertex in a simple connected graph $\Gamma$, finds **all** shortest paths to any vertex $y$ in $\Gamma$.

* Say that a partition $\lambda$ **covers** a partition $\mu$ if $\mu$ can be obtained by decreasing a part of $\lambda$ by $1$.  Denote by $\geq$ the reflexive and transitive closure of the covering relation.  The **Young lattice** $Y(\lambda)$ is the graph on all partitions $\mu \leq \lambda$ with the covering relation as edges.  For a partition $\lambda$, compute its Young lattice $Y(\lambda)$ as orbit of $\lambda$ under a suitable action.

* A **standard Young diagram** (SYT) of shape $\lambda$ is a shortest path from the empty partition $\emptyset$ to $\lambda$ in the Young lattice $Y(\lambda)$. For a given partition $\lambda$, compute all SYTs of shape $\lambda$ as a set of shortest paths.

In [ ]:
Read("syt.g");
l:= standardYTs([3,1]);
PrintArray(tableau_path(l[3]));

* A **round trip** of shape $\lambda$ is a shortest path from $\emptyset$ to $\lambda$ and back along a 
(possibly different) shortest path. Verify that the total number of round trips to all the partitions $\lambda$ of $4$ is $24$.  What is the general formula for the total number of round trips for all partitions of $n$? Why?

In [ ]:
Read("syt.g");
Sum(partitions(4), x-> Length(standardYTs(x))^2);

* Implement the **group membership test** $x \in G$, using stabilizers.

* ($*$) In practice, the number of Schreier generators of the stabilizers in the chain can grow very fast, in larger examples.  The **Schreier-Sims** algorithm intertwines orbit calculations and membership tests to keep the number of necessary generators small.  Implement such a strategy.

* ($**$) Compute the order of the Rubik's cube group
```gap
cube := Group(
( 1, 3, 8, 6)( 2, 5, 7, 4)( 9,33,25,17)(10,34,26,18)(11,35,27,19),
( 9,11,16,14)(10,13,15,12)( 1,17,41,40)( 4,20,44,37)( 6,22,46,35),
(17,19,24,22)(18,21,23,20)( 6,25,43,16)( 7,28,42,13)( 8,30,41,11),
(25,27,32,30)(26,29,31,28)( 3,38,43,19)( 5,36,45,21)( 8,33,48,24),
(33,35,40,38)(34,37,39,36)( 3, 9,46,32)( 2,12,47,29)( 1,14,48,27),
(41,43,48,46)(42,45,47,44)(14,22,30,38)(15,23,31,39)(16,24,32,40) 
);;
```

In [ ]:
Read("sims.g");
Read("cube.g");
size_sims(cube);